In [11]:
import pandas as pd
import numpy as np
import locale
locale.setlocale( locale.LC_ALL, '' )

'English_United States.1252'

In [12]:

distances = pd.read_csv('dist.csv')
depos = pd.read_csv('depos.csv')
repairs_costs = pd.read_csv('repairs_costs.csv')
stations = pd.read_csv('stations.csv')
vagons_repairs = pd.read_csv('vagons_repairs.csv').dropna()
vagons_ways = pd.read_csv('vagons_ways.csv')

all_data = [distances, depos, repairs_costs, stations, vagons_repairs, vagons_ways]
table_names = "distances, depos, repairs_costs, stations, vagons_repairs, vagons_ways".split(", ")

In [13]:
for i in range(len(table_names)):
    print(table_names[i].center(100).upper(), end='\n\n')
    print(all_data[i].head(), end='\n\n\n')

                                             DISTANCES                                              

   station_id      name  depo_id           depo_name  distance_to_depo
0           1  Сосновка        1      Депо-Ленинское             10077
1           1  Сосновка        2  Депо-Комсомольское             10436
2           1  Сосновка        3     Депо-Молодежное             10522
3           2   Липовка        1      Депо-Ленинское              2084
4           2   Липовка        2  Депо-Комсомольское              3789


                                               DEPOS                                                

   id           depo_name
0   1      Депо-Ленинское
1   2  Депо-Комсомольское
2   3     Депо-Молодежное


                                           REPAIRS_COSTS                                            

   id                                 repair_name  repair_cost
0   1                          Капитальный ремонт     94000000
1   2                            Д

In [14]:


def getRepairCostForType(x, closest_depos):
    return np.ceil(closest_depos.iloc[x - 1]['distance_to_depo'] / 500) * TARIF*2



In [15]:
TARIF = 10000 * 500 *500


In [16]:

def optimize_please(wagon):
    closest_depos = distances.loc[distances.groupby('name', as_index=False, sort=False)['distance_to_depo'].idxmin()]

    #Оптимальное депо для каждого города
    array_of_closest_depos = np.array(closest_depos["distance_to_depo"])

    # путевой лист
    ways = vagons_ways[vagons_ways['car_num'] == wagon].reset_index()
    putevoi_list = np.array(ways['station_id'])

    #Даты плановых ремонтов
    type_1 = vagons_repairs[vagons_repairs["car_num"] == wagon]["std_kap"].iloc[0]
    type_2 = vagons_repairs[vagons_repairs["car_num"] == wagon]["std_ppr"].iloc[0]
    type_3 = vagons_repairs[vagons_repairs["car_num"] == wagon]["std_dep"].iloc[0]
    type_4 = vagons_repairs[vagons_repairs["car_num"] == wagon]["std_vogi"].iloc[0]
    type_1, type_2, type_3, type_4

    planned_towns = []
    planned_towns_indicies = []

    date_for_type_1 = ways[ways["date"] > type_1]
    id_planned_town = date_for_type_1.iloc[0].name -1
    planned_towns.append(ways.iloc[date_for_type_1.iloc[0].name -1]["station_id"])
    planned_towns_indicies.append(id_planned_town)

    date_for_type_2 = ways[ways["date"] > type_2]
    id_planned_town = date_for_type_2.iloc[0].name -1
    planned_towns.append(ways.iloc[date_for_type_2.iloc[0].name -1]["station_id"])
    planned_towns_indicies.append(id_planned_town)

    date_for_type_3 = ways[ways["date"] > type_3]
    id_planned_town = date_for_type_3.iloc[0].name -1
    planned_towns.append(ways.iloc[date_for_type_3.iloc[0].name -1]["station_id"])
    planned_towns_indicies.append(id_planned_town)

    date_for_type_4 = ways[ways["date"] > type_4]
    id_planned_town = date_for_type_4.iloc[0].name -1
    planned_towns.append(ways.iloc[date_for_type_4.iloc[0].name -1]["station_id"])
    planned_towns_indicies.append(id_planned_town)

    planned_towns = np.array(planned_towns)
    planned_towns


    # Стоимость планового ремонта
    planned_costs = np.apply_along_axis(lambda x: np.ceil(closest_depos.iloc[x - 1]['distance_to_depo'] / 500) * TARIF*2,
                                        axis=0, arr=planned_towns)
    planned_costs +=  repairs_costs['repair_cost']
    planned_costs = planned_costs.sum()

    #Количество подготовок
    number_of_prepares = len(set(planned_towns_indicies))
    planned_costs += 3*1000*1000 * number_of_prepares
    planned_costs


    current_min = float("inf")

    # 4 в ложенных цiкла (secret technique)
    output_costs = [] # 10 000 values
    output_combinations = []
    for i in [10,9,2,6,3,4,5,8,7,1]: # по возрастанию стоимости
        cost_type_1 = 0
        optimal_station_1 = -1
        planned_town_1 = planned_towns[0] # тип города, в котором будет запланированный ремонт поломки 1
        id_planned_town_1 = planned_towns_indicies[0] # индекс города, в котором будет запланированный ремонт поломки 1
        for station in range(id_planned_town_1, 0, -1):
            if (putevoi_list[station] == i): # мы нашли город, в котором будем оптимально чинить поломку 1
                cost_type_1 = getRepairCostForType(putevoi_list[station], closest_depos)
                cost_type_1 += (id_planned_town_1 - station)*24*TARIF
                cost_type_1 +=   repairs_costs['repair_cost'].iloc[0]
                optimal_station_1 = station
                break

        if (cost_type_1 > current_min):
            continue

        for j in [10,9,2,6,3,4,5,8,7,1]:
            cost_type_2 = 0
            optimal_station_2 = -1
            planned_town_2 = planned_towns[1] # тип города, в котором будет запланированный ремонт поломки 2
            id_planned_town_2 = planned_towns_indicies[1] # индекс города, в котором будет запланированный ремонт поломки 2
            for station in range(id_planned_town_2, 0, -1):
                if (putevoi_list[station] == j): # мы нашли город, в котором будем оптимально чинить поломку 2
                    cost_type_2 = getRepairCostForType(putevoi_list[station], closest_depos)
                    cost_type_2 += (id_planned_town_2 - station)*24*TARIF
                    cost_type_2 +=   repairs_costs['repair_cost'].iloc[1]
                    optimal_station_2 = station
                    break
            if (cost_type_1 + cost_type_2 > current_min):
                continue
                
            for k in [10,9,2,6,3,4,5,8,7,1]:
                cost_type_3 = 0
                optimal_station_3 = -1
                planned_town_3 = planned_towns[2] # тип города, в котором будет запланированный ремонт поломки 3
                id_planned_town_3 = planned_towns_indicies[2] # индекс города, в котором будет запланированный ремонт поломки 3
                for station in range(id_planned_town_3, 0, -1):
                    if (putevoi_list[station] == k): # мы нашли город, в котором будем оптимально чинить поломку 3
                        cost_type_3 = getRepairCostForType(putevoi_list[station], closest_depos)
                        cost_type_3 += (id_planned_town_3 - station)*24*TARIF
                        cost_type_3 +=   repairs_costs['repair_cost'].iloc[2]
                        optimal_station_3 = station
                        break

                if (cost_type_1 + cost_type_2  + cost_type_3 > current_min):
                    continue
                for m in [10,9,2,6,3,4,5,8,7,1]:
                    cost_type_4 = 0
                    planned_town_4 = planned_towns[3] # тип города, в котором будет запланированный ремонт поломки 4
                    id_planned_town_4 = planned_towns_indicies[3] # индекс города, в котором будет запланированный ремонт поломки 4
                    for station in range(id_planned_town_4, 0, -1):
                        if (putevoi_list[station] == m): # мы нашли город, в котором будем оптимально чинить поломку 4
                            cost_type_4 = getRepairCostForType(putevoi_list[station], closest_depos)
                            cost_type_4 += (id_planned_town_4 - station)*24*TARIF
                            cost_type_4 +=   repairs_costs['repair_cost'].iloc[3]
                            stations = [optimal_station_1, optimal_station_2, optimal_station_3, station]
                            appendix  = 3*1000*1000*len(set(stations))
                            output_costs.append(cost_type_1 + cost_type_2+cost_type_3+cost_type_4 + appendix)
                            output_combinations.append(stations)
                            if (current_min > cost_type_1 + cost_type_2+cost_type_3+cost_type_4):
                                current_min = cost_type_1 + cost_type_2+cost_type_3+cost_type_4
                            break


    min_index_optimal = np.argmin(output_costs)

    saved = locale.currency(planned_costs - min(output_costs), grouping=True)

    output_combinations[min_index_optimal]

    planned_towns_indicies
    if (planned_costs != min(output_costs)):
        print("Saved", saved, "combinations", output_combinations[min_index_optimal], "planned", planned_towns_indicies)
    return [min(output_costs), planned_costs, output_combinations[min_index_optimal],  planned_towns_indicies]

In [17]:

TARIF = 1000 * 500

optimal_repairs = []
counter = 1
for i in vagons_repairs['car_num']:
    optimal_cost, planned_cost, optimal_towns, planned_towns = optimize_please(i)
    optimal_repairs.append([i, optimal_cost, planned_cost, optimal_towns, planned_towns])
    counter += 1
    if (counter % 100 == 0):
        print("reached mark of", counter, "elements")


Saved $2,000,000.00 combinations [98, 64, 63, 64] planned [98, 64, 63, 65]
reached mark of 100 elements
Saved $1,000,000.00 combinations [95, 79, 124, 170] planned [95, 79, 125, 170]
reached mark of 200 elements
Saved $1,000,000.00 combinations [76, 49, 124, 49] planned [76, 49, 125, 49]
Saved $1,000,000.00 combinations [94, 79, 124, 170] planned [94, 79, 125, 170]
reached mark of 300 elements
Saved $1,000,000.00 combinations [190, 68, 68, 124] planned [190, 68, 68, 125]
reached mark of 400 elements
reached mark of 500 elements
Saved $1,000,000.00 combinations [201, 78, 79, 124] planned [201, 78, 79, 125]
Saved $1,000,000.00 combinations [67, 59, 105, 124] planned [67, 59, 105, 125]
Saved $1,000,000.00 combinations [183, 61, 73, 124] planned [183, 61, 73, 125]
reached mark of 600 elements
reached mark of 700 elements
reached mark of 800 elements
Saved $1,000,000.00 combinations [80, 79, 124, 170] planned [80, 79, 125, 170]
Saved $2,000,000.00 combinations [157, 64, 64, 86] planned [157

In [18]:
len(optimal_repairs)

15035

In [19]:
all_saved = 0
for i in optimal_repairs:
    all_saved += i[2] - i[1]
all_saved

482000000.0

In [21]:
optimal_repairs

[[57463085, 179000000.0, 179000000.0, [100, 55, 70, 66], [100, 55, 70, 66]],
 [57456220, 178000000.0, 178000000.0, [100, 66, 71, 66], [100, 66, 71, 66]],
 [57453656, 185000000.0, 185000000.0, [99, 61, 68, 66], [99, 61, 68, 66]],
 [57463507, 187000000.0, 187000000.0, [99, 64, 52, 66], [99, 64, 52, 66]],
 [57463309, 209000000.0, 209000000.0, [98, 51, 61, 65], [98, 51, 61, 65]],
 [57463341, 198000000.0, 198000000.0, [99, 54, 52, 66], [99, 54, 52, 66]],
 [57463275, 231000000.0, 231000000.0, [98, 56, 56, 65], [98, 56, 57, 65]],
 [57463317, 207000000.0, 207000000.0, [98, 60, 67, 65], [98, 60, 67, 65]],
 [57457038, 191000000.0, 191000000.0, [59, 58, 104, 149], [59, 58, 104, 149]],
 [57463523, 221000000.0, 221000000.0, [98, 65, 63, 65], [98, 65, 63, 65]],
 [57463382, 213000000.0, 213000000.0, [98, 61, 77, 65], [98, 61, 77, 65]],
 [57463572, 208000000.0, 208000000.0, [99, 63, 65, 65], [99, 63, 65, 65]],
 [57463606, 211000000.0, 213000000.0, [98, 64, 63, 64], [98, 64, 63, 65]],
 [57463481, 20700

In [117]:
web = []
for i in optimal_repairs:
    entry = []
    wagon = i[0]# номер вагона
    entry.append(wagon) 
    ways = vagons_ways[vagons_ways['car_num'] == wagon].reset_index()
    for j in range(4): # four types of repair
        city_to_do_type_j = i[3][j]
        entry.append(ways.iloc[city_to_do_type_j]['date']) # date of typej
    profit = i[2] - i[1]
    entry.append(profit) # разница между деньгами
    preparings_saved = (len(set(i[4])) - len(set(i[3]))) * 3*1000*1000
    entry.append(preparings_saved) #подготовка: появившееся количесвто комбинаций
    entry.append(profit - preparings_saved) # логистика
    for j in range(4):#need update
        if(i[3][j] != i[4][j]):
            entry.append('g')
        else:
            entry.append('b')
    web.append(entry)
    


In [44]:
# вагон, дата капитального, дата .. , разница между деньгами, подготовка и логистика

In [46]:
web_file = open('web_file', 'w+')
for i in web:
    web_file.write(', '.join([str(e) for e in i]))
    web_file.write("\r\n")
web_file.close()

In [122]:
for i in range(2018, 2030):
    for j in range(1, 13):
        print("starting", str(i),"-",str(j))
        if (i == 2018 and j < 5):
            pass
        tmp_date = str(i) + '-' + str(j).zfill(2)
        ij_file = []
        for k in web:
            if (k[1].startswith(tmp_date+"-") or k[2].startswith(tmp_date+"-") or k[3].startswith(tmp_date+"-") or k[4].startswith(tmp_date+"-")):
                ij_file.append(k)
        f = open(tmp_date +".txt", 'w+')
        for k in ij_file:
            f.write(', '.join([str(e) for e in k]))
            f.write("\r\n")
        f.close()
        

starting 2018 - 1
starting 2018 - 2
starting 2018 - 3
starting 2018 - 4
starting 2018 - 5
starting 2018 - 6
starting 2018 - 7
starting 2018 - 8
starting 2018 - 9
starting 2018 - 10
starting 2018 - 11
starting 2018 - 12
starting 2019 - 1
starting 2019 - 2
starting 2019 - 3
starting 2019 - 4
starting 2019 - 5
starting 2019 - 6
starting 2019 - 7
starting 2019 - 8
starting 2019 - 9
starting 2019 - 10
starting 2019 - 11
starting 2019 - 12
starting 2020 - 1
starting 2020 - 2
starting 2020 - 3
starting 2020 - 4
starting 2020 - 5
starting 2020 - 6
starting 2020 - 7
starting 2020 - 8
starting 2020 - 9
starting 2020 - 10
starting 2020 - 11
starting 2020 - 12
starting 2021 - 1
starting 2021 - 2
starting 2021 - 3
starting 2021 - 4
starting 2021 - 5
starting 2021 - 6
starting 2021 - 7
starting 2021 - 8
starting 2021 - 9
starting 2021 - 10
starting 2021 - 11
starting 2021 - 12
starting 2022 - 1
starting 2022 - 2
starting 2022 - 3
starting 2022 - 4
starting 2022 - 5
starting 2022 - 6
starting 2022 - 